<a href="https://colab.research.google.com/github/GarnetKangSB/MultiCampus_2020NLP_BOK_3/blob/master/CrawlingPDF_using_tika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget http://apache.mirror.cdnetworks.com/tika/tika-server-1.19.jar

--2020-07-28 04:58:27--  http://apache.mirror.cdnetworks.com/tika/tika-server-1.19.jar
Resolving apache.mirror.cdnetworks.com (apache.mirror.cdnetworks.com)... 14.0.101.165
Connecting to apache.mirror.cdnetworks.com (apache.mirror.cdnetworks.com)|14.0.101.165|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-07-28 04:58:27 ERROR 404: Not Found.



In [ ]:
# https://nearman.tistory.com/entry/8-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-PDF%ED%8C%8C%EC%9D%BC-%EC%9D%BD%EC%96%B4%EC%98%A4%EA%B8%B0-PDF%ED%8C%8C%EC%9D%BC%EC%97%90%EC%84%9C-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EB%82%B4%EC%9A%A9-%ED%8C%8C%EC%8B%B1

In [ ]:
pip install tika 

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=cc60b1c007afe02abc76d0d28d4c53e022add3c9c4c589eb817375222bdc3e0e
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
from tika import parser

In [ ]:
# poppler 
!apt-get install poppler-utils 
!apt-get update
!apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 2s (102 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Get:1 https://cloud.r-project.org/bin/linux/ub

In [ ]:
# pdftotext 
!pip install pdftotext

     |████████████████████████████████| 122kB 2.6MB/s 
  Created wheel for pdftotext: filename=pdftotext-2.1.4-cp36-cp36m-linux_x86_64.whl size=54067 sha256=c80f1725eadbdaa5501faf83463c1a813af95eea3359e65cb993106234c3cdc9
  Stored in directory: /root/.cache/pip/wheels/88/c3/62/c1ef7f738f8c0c1ac67cbe63786cae734d555c5077088036f0
Successfully built pdftotext


In [ ]:
######### 네이버 채권 분석 

######### Import #########

# Crawling
from bs4 import BeautifulSoup
import requests
import re
import copy

# PDF library
#import pdftotext 

# CSV
import csv

# to Create Folder
import os

# Save File
# from URL
from urllib.request import urlretrieve



######### Main #########

# Initialize

# Create Folder 

LootPath = "/content/drive/My Drive/Analysis"

try:
    if not (os.path.isdir(LootPath)):
        os.makedirs(os.path.join(LootPath))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
"""
try:
    if not (os.path.isdir(LootPath + "/Text")):
        os.makedirs(os.path.join(LootPath + "/Text"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
"""  
        
        

# Data Crawling
        
page_idx = 0 # Page Index 
pdf_idx = 0 # PDF File Index
error_count = 0 #Check FXXXXXX Error
day_list_check = []

while True:
    
    # Page Index 
    page_idx += 1
    # PDF File Index
    pdf_idx += 1
    
    # Get new Data List
    try:
        URL = "https://finance.naver.com/research/debenture_list.nhn?&page={}".format(page_idx)
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser") 
    # End of List
    except:
        break
    
    # 
    day_list=[] # date of PDF file
    pdf_list=[] # url of PDF file
    
    # Get Date of Data
    dates = soup.select('div.box_type_m td.date')
    days = re.findall("[0-9][0-9].[0-9][0-9].[0-9][0-9]",str(dates))
    for day in days:
        # remove '.' 
        day_list.append(day.replace(".",""))
        
    # to End Squence
    # further iterates do same work
    # check Data if its same to previous one
    ### maybe : if exceed maxium page number, return last page
    ### maybe : if exceed maxium page number, return None
    if day_list == day_list_check: #why 'is' doesn't work ?
        break
    else:
        day_list_check = copy.deepcopy(day_list)
    
    # Check Process
    ### Erase later
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page :",page_idx,">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(">>> from" + day_list[0] + "~~~")
    
    # Check Duration
    # Too recent
    if all(int(day) > 180101 for day in day_list):
        print(">>> new : PASS")
        continue
    # Too past
    if all(int(day) < 40401 for day in day_list):
        print(">>> old : PASS")
        break
    
    # Get URL of Data
    files = soup.find_all(class_='file')
    pdfs = re.findall("(?<=a href=\")[^ ]+pdf",str(files))
    for pdf in pdfs:
        pdf_list.append(pdf)
    
    # another Methodology
    # to set file to 'UTF-8' format
    """setting = "# -*- encoding: utf-8 -*-"
    with open(path,'w',newline='') as data_csv:
                    write_data = csv.writer(data_csv) 
                    write_data.writerow(setting)"""

    # Set Path
    path_CSV = LootPath + '/Analysis_CSV{}.csv'.format(pdf_idx)
    #path_TXT = LootPath + '/Analysis_TXT{}.txt'.format(pdf_idx)
    
    with open(path_CSV,'a',newline='') as data_csv:
        write_data_CSV = csv.writer(data_csv)
        # make data pretty ***
        tmp_list = list()
        tmp_list.append("Page_idx")
        tmp_list.append("PDF_No")
        tmp_list.append("Date")
        tmp_list.append("content")
        write_data_CSV.writerow(tmp_list)
        
    # Get Proper Data
    ### Check Date
    for idx, pdf in enumerate(pdf_list):
        if 40331 < int(day_list[idx]) < 180101:
            
            # from URL
            # save PDF as file
            urlretrieve(pdf, LootPath + '/tmpAnalysis.pdf')
            
            # PDF to CSV
            # Read PDF File    
            with open(LootPath + '/tmpAnalysis.pdf','rb') as file:
            #file = open('./Analysis/Analysis.pdf','rb')
                tmp_pdf = parser.from_file(LootPath + '/tmpAnalysis.pdf')

                tmp = tmp_pdf["content"]
                
                # Read by Page
                #for pn in range(len(fileReader)):

                # Check Process
                ### Erase later
                print("Page:",page_idx,"Date:",day_list[idx],"File:",idx)#,"Page:",pn)

                # Error : poppler error creating page
                # wtf
                try:
                    # Get one Page of PDF

                    # Do something
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                        
                # Error Check
                except Exception as e:
                    error_count += 1
                    print(e,"error_count :",error_count)
                    pass
                        
                    

                    #######################################################
                    # 위의껄로 하면 에러 수는 적어짐
                    # 아래껄로 하면 파일 읽을 때 주의해야 함
                    #######################################################
                    
                    # 위 #
                    # -1 : Kernel ???
                    # 'utf-8' : Set Format
                    # without this
                    # Error : 'cp949' codec can't encode character '\u201a' in position 1184: illegal multibyte sequence
                    """   
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path,'a',-1,'utf-8',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp)
                        
                    # open TXT file as 'Append' mode
                    # to insert new Data
                    with open(path_TXT,'a',-1,'utf-8',newline='') as data_txt:

                        # Write a Page to TXT
                        data_txt.write(tmp)
                    """
                    
                # 아래 #
                # open CSV file as 'Append' mode
                # to insert new Data
                with open(path_CSV,'a',newline='') as data_csv:

                    # Make Writer
                    write_data_CSV = csv.writer(data_csv) 

                    # make data pretty ***
                    tmp_list = list()
                    tmp_list.append(page_idx)
                    tmp_list.append(idx)
                    tmp_list.append(day_list[idx])
                    tmp_list.append(tmp)
                    
                    # Prepare to Error
                    # in specific pdf
                    # Error : poppler error creating page
                    # wtf
                    try:
                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp_list)
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass
                    """
                    with open(path_TXT,'a',newline='') as data_txt:

                        # Prepare to Error
                        try:
                            # Write a Page to TXT
                            data_txt.write(tmp_list)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass"""

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 1 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200731~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 2 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200624~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 3 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200528~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 4 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200420~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 5 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200325~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 6 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200305~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 7 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from200128~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 8 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from191206~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 9 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from191028~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 10 >>>>>>>>>>>>>>>>>>>>>>>

# 새 섹션

In [ ]:
######### B.O.K 금통위 의사록 

######### Import #########

# Crawling
from bs4 import BeautifulSoup
import requests
import re
import copy

# CSV
import csv

# to Create Folder
import os

# Save File
# from URL
from urllib.request import urlretrieve



######### Main #########

LootPath = "/content/drive/My Drive/BoK"

# Initialize
# Create Folder
try:
    if not (os.path.isdir(LootPath)):
        os.makedirs(os.path.join(LootPath))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
"""
try:
    if not (os.path.isdir(LootPath + "/Text")):
        os.makedirs(os.path.join(LootPath + "/Text"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
"""        
        
# Data Crawling
        
page_idx = 0 # Page Index 
pdf_idx = 0 # PDF File Index
error_count = 0 # Check FXXXXXX Error
day_list_check = []

while True:
    
    # Page Index 
    page_idx += 1
    # PDF File Index
    pdf_idx += 1
    
    
    
    # Get new Data List
    try:
        URL = "http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}".format(page_idx)
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser") 
    # End of List
    except:
        break
    
    # 
    day_list=[]
    pdf_list=[]
    
    # Get Date of Data
    dates = soup.select('span.titlesub')
    days = re.findall("[0-9][0-9][0-9][0-9].[0-9]+.[0-9]+",str(dates))
    
    for day in days:
        # Fill 0 to month
        if day[6] == '.':
            day = day[:5]+"0"+day[5:]
        # Fill 0 to day
        try:
            day[9]
        except:
            day = day[:7]+"0"+day[8]
        # remove '.' 
        day_list.append(day.replace(".",""))
        
    # to End Squence
    # further iterates do same work
    # check Data if its same to previous one
    ### maybe : if exceed maxium page number, return last page
    ### maybe : if exceed maxium page number, return None
    if day_list == day_list_check: #why 'is' doesn't work ?
        break
    else:
        day_list_check = copy.deepcopy(day_list)
    
    # Check Process
    ### Erase later
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page :",page_idx,">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(">>> from" + day_list[0] + "~~~")
    
    # Check Duration
    # Too recent
    if all(int(day) > 20180101 for day in day_list):
        print(">>> new : PASS")
        continue
    # Too past
    if any(int(day) < 20040401 for day in day_list):
        print(">>> old : PASS")
        break
    
    # Get URL of Data
    #files = soup.find_all('div .fileGoupBox')
    files = soup.find_all(class_='fileGoupBox')
    
    pdfs = re.findall("(?<=a href=\")[^ ]+(?=\")",str(files))
    name = re.findall("(?<=title=\").+\.[a-z][a-z][a-z]",str(files))
    
    # Get PDF only
    for idx in range(len(pdfs)):
        if (name[idx][-3:] == "pdf"):
            # Get URL of Data
            # Revise URL
            # 아니 저게 왜 들어가있는거지
            pdf_list.append(pdfs[idx].replace("amp;",""))
    
    # another Methodology
    # to set file to 'UTF-8' format
    """setting = "# -*- encoding: utf-8 -*-"
    with open(path,'w',newline='') as data_csv:
                    write_data = csv.writer(data_csv) 
                    write_data.writerow(setting)"""

    # Set Path
    path_CSV = LootPath + '/BoK_CSV{}.csv'.format(pdf_idx)
    #path_TXT = LootPath + '/BoK_TXT{}.txt'.format(pdf_idx)
    
    with open(path_CSV,'a',newline='') as data_csv:
        write_data_CSV = csv.writer(data_csv)
        # make data pretty ***
        tmp_list = list()
        tmp_list.append("Page_idx")
        tmp_list.append("PDF_No")
        #tmp_list.append("Page_No")
        tmp_list.append("Date")
        tmp_list.append("content")
        write_data_CSV.writerow(tmp_list)

    # Get Proper Data
    ### Check Date
    for idx, pdf in enumerate(pdf_list):
        if 20040331 < int(day_list[idx]) < 20180101:
            
            URL="http://www.bok.or.kr"
            # from URL
            # save PDF as file
            urlretrieve(URL+pdf, LootPath + '/tmpBoK.pdf')
            
            # PDF to CSV
            # Read PDF File    
            with open(LootPath + '/tmpBoK.pdf','rb') as file:
            #file = open('./BoK/Analysis.pdf','rb')
                
                tmp_pdf = parser.from_file(LootPath + '/tmpBoK.pdf')
                #parser.
                print(tmp_pdf["content"])
                #fileReader = pdftotext.PDF(file)
            

                # Read by Page
                #for pn in range(len(fileReader)):

                # Check Process
                ### Erase later
                print("Page:",page_idx,"Date:",day_list[idx],"File:",idx)#,"Page:",pn)

                # Error : poppler error creating page
                # wtf
                try:
                    # Get one Page of PDF
                    tmp = tmp_pdf["content"]

                    # Do something
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                    tmp=tmp.replace("  "," ")
                    tmp=tmp.replace("\t\t","\t")
                    tmp=tmp.replace("\n\n","\n")
                    
                # Error Check
                except Exception as e:
                    error_count += 1
                    print(e,"error_count :",error_count)
                    pass
                        
                    

                #######################################################
                # 위의껄로 하면 에러 수는 적어짐
                # 아래껄로 하면 파일 읽을 때 주의해야 함
                #######################################################
                
                # 위 #
                # -1 : Kernel ???
                # 'utf-8' : Set Format
                # without this
                # Error : 'cp949' codec can't encode character '\u201a' in position 1184: illegal multibyte sequence
                """   
                # open CSV file as 'Append' mode
                # to insert new Data
                with open(path,'a',-1,'utf-8',newline='') as data_csv:

                    # Make Writer
                    write_data_CSV = csv.writer(data_csv) 

                    # Write a Page to CSV
                    write_data_CSV.writerow(tmp)
                    
                # open TXT file as 'Append' mode
                # to insert new Data
                with open(path_TXT,'a',-1,'utf-8',newline='') as data_txt:

                    # Write a Page to TXT
                    data_txt.write(tmp)
                """
                
                # 아래 #
                # open CSV file as 'Append' mode
                # to insert new Data
                with open(path_CSV,'a',newline='') as data_csv:

                    # Make Writer
                    write_data_CSV = csv.writer(data_csv) 

                    # Prepare to Error
                    # in specific pdf
                    # Error : poppler error creating page
                    # wtf
                    
                    # make data pretty ***
                    tmp_list = list()
                    tmp_list.append(page_idx)
                    
                    tmp_list.append(idx)
                    #tmp_list.append(pn)
                    tmp_list.append(day_list[idx])
                    tmp_list.append(tmp)
                    

                    try:
                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp_list)
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass
                """
                with open(path_TXT,'a',newline='') as data_txt:

                    # Prepare to Error
                    try:
                        # Write a Page to TXT
                        data_txt.write(tmp_list)
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass"""
                    
                    

# PDF 링크
# http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000018397&fileSn=2


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 1 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20200624~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 2 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20200117~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 3 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20190531~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 4 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20181206~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 5 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20180426~~~


2020-07-31 16:17:42,689 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-07-31 16:17:43,958 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-07-31 16:17:44,448 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.



- 12 -

가로 인하여 주요 선진국 및 중국의 경제성장이 지체될 우려가 있어 우리나라의 

수출상황을 낙관할 수만은 없으며 실질무역손실이 큰 폭으로 증가하면서 국내 총

소득 증가율이 정체되어 소비회복을 제약할 가능성이 있음; 셋째, 통화정책이 중

립보다는 완화 기조를 지속함에 따라 시중 유동성이 다소 과잉을 나타내고 있는 

것으로 보이며 이는 경기회복이 지연되면서 자금의 단기부동화로 이어져 부동산

가격을 상승시키고 금융시장 및 거시경제의 불안정성과 불확실성을 높이고 있음. 

부동산투기가 심각할 정도로 확산된다고 판단될 경우 과다한 비용을 지불하지 않

도록 사전 대응책이 마련되어야 하겠으나 정부가 부동산시장 안정 종합대책을 마

련하고 있어 그 내용과 효과를 지켜본 후 통화정책적 대응을 하는 것이 바람직하

다고 생각됨; 넷째, 미국이 정책금리를 지속적으로 상향조정하면서 해외 자본유출 

가능성이 높아지고 있음. 급격한 자본유출은 환율을 상승시켜 물가상승압력으로 

작용할 수 있으며 금융시장을 불안정하게 하고 통화관리에도 부담이 될 수 있어, 

금융의 범세계화 그리고 이에 따른 금융의 세계적인 통합화 추세를 수용할 수 있

는 통화관리체제를 구축하는 것이 필요할 것으로 생각됨; 다섯째, 금년 들어 시중

자금의 배분이 금리 변화에 취약한 중소기업과 가계를 중심으로 크게 확대되고 

있어 통화신용정책에 대한 민감도가 예전보다 높아질 것으로 보이기 때문에, 통

화신용정책이 미칠 효과, 우리 경제의 양극화 문제 등을 고려하여 신중한 정책결

정이 이루어져야 할 것으로 생각됨

한편 일부 위원은 다음과 같은 점을 종합적으로 고려할 때 금리인상을 고

려할 시기에 이르 다고 보이나, 통화정책이 일관성 있는 방향성을 가지고 움직

여야 된다는 점을 고려할 때 현재가 그 같은 방향을 설정할 시점인가에 대해서는 

아직 확신이 없기 때문에 이달에는 콜금리 목표를 현 수준에서 유지하는 것이 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("end?")

end?
